# Analysis of the single assessment


In [ ]:
import os
import numpy as np
import pandas as pd   # module to work with data in DataFrames.
import seaborn as sns  # module to plot DataFrames in an easy way
import matplotlib.pyplot as plt
from pathlib import Path
import scipy.constants as sc
import scipy.stats as stats

Seaborn setup.

In [ ]:
# seaborn comes with default presets (called 'contents') for figure's aesthetics, 
# such as the font size, figure size etc.

sns.set_context('paper') # try: 'talk', 'poster', 'notebook'
sns.set_theme(style="ticks", palette="pastel")

# Set global color scheme for historic and modern images.
sns_palette_historic = ["m"]
sns_palette_modern = ["g"]

# Plot size and y limits.
sns_plot_size = (12*sc.golden_ratio, 12)
sns_half_plot_size = (4*sc.golden_ratio, 12)
sns_ylim = (-0.5, 1)

color_blue = '#4E92FF'

def set_hline(plot):
    plot.axhline(0.0, color=color_blue, linewidth=2.0)

## Get the result files

In [ ]:
# Get current result files.
result_dir = Path('../nines/results').absolute()
result_file_names = os.listdir(result_dir)
result_file_names = list(filter(lambda name: str.startswith(name, 'result'), result_file_names))

single_result_files_paths = []
for name in result_file_names:
    single_result_files_paths.append(result_dir / name)

num_observers = len(single_result_files_paths)
print(f"Number observer: {num_observers}")

## Create dataframe

In [ ]:
df = pd.DataFrame()
for index, path in enumerate(single_result_files_paths):
    df_tmp = pd.read_csv(path)
    df_tmp['observer'] = index
    df = pd.concat((df, df_tmp))

df.columns = ['test_image', '0', '1', '2', '3', '4', '5', '6', '7', '8', 'choosen', 'observer']


Add short ID of the image to the dataframe.

In [ ]:
df['id'] = [s.split('_')[0] for s in df['test_image'].tolist()]

Sort the dataframe by id of the images.

In [ ]:
df = df.sort_values(by='id', ascending=True, key=lambda col: [int(img[1:]) for img in col])

Add type of the image in new column. Historic or modern (H, I).

In [ ]:
df['type'] = [s[0] for s in df['id'].tolist()]

Some observers may have seen more images than planned for.
These images are removed.

In [ ]:
# Historic image ids:
list_of_used_images = [f"H{index}" for index in range(30)]
# Modern images ids:
list_of_used_images += [f"I{index:02d}" for index in [1, 3, 4, 6, 7, 8, 12, 13, 14, 15 ,16, 20, 21, 22, 23]]

assert len(list_of_used_images) == 30 + 15

df = df[df["id"].isin(list_of_used_images)]

### Normalise chroma values
The chroma values are normalised to the range [0, 1].
Subtract all values to locate algorithm result factor at 0.0 in plots.

In [ ]:
df["normalise"] = [(entry[0] / entry[1]) - (1 / entry[1]) for entry in list(zip(df["choosen"].tolist(), df["8"].tolist()))]

## Sanity checks
We expect 30 historic and 15 modern images used during tests.
Each result file  from each observer shall be 30 + 15 images long.

In [ ]:
# sanity checks: how many groups do you expect per categorical variable?
assert len(df['test_image'].unique()) == 30 + 15

# Check number of observers.
assert len(df['observer'].unique()) == len(result_file_names)

# Check length of result from each observer.
for observer_id in df['observer'].unique():
    assert len(df[df['observer'] == observer_id]) == 30 + 15
    assert len(df[df['observer'] == observer_id]['test_image'].unique()) == 30 + 15

# Check number of historic and modern images.
assert len(df[df['type'] == "H"]) == 30 * num_observers
assert len(df[df['type'] == "I"]) == 15 * num_observers

## Dataframe

In [ ]:
df

Max factor used throughout all images.

In [ ]:
max_factor = np.ceil(max(df[[str(i) for i in range(9)]].max(axis=0)))


In [ ]:
df[df["type"] == "H"]["normalise"]

### Historic plots

Boxplot

In [ ]:
g = sns.catplot(
    x='id',
    y='normalise',
    data=df[df["type"] == "H"],
    kind='box',
    ci='sd',
    palette=sns_palette_historic
)
set_hline(g.axes[0][0])
g.fig.set_size_inches(sns_plot_size)
g.set(ylim = sns_ylim)

g.set_axis_labels("Image", "Chroma factor")
g.set(title="Niner assessment: Historic images")

sns.despine()

### Modern plots

In [ ]:
g = sns.catplot(
    x='id',
    y='normalise',
    data=df[df["type"] == "I"],
    kind='box',
    ci='sd', 
    palette=sns_palette_modern
)
set_hline(g.axes[0][0])

g.fig.set_size_inches(sns_plot_size)

g.set(ylim = sns_ylim)

g.set_axis_labels("Image", "Chroma factor")
g.set(title="Niner assessment: Modern images")
sns.despine()

# Plots for both image groups

Get mean of normalised choosen factor for each image. Stored in own dataframe.

In [ ]:
means = []
for img in df[df["type"] == "H"]["id"].unique():
    means.append((img, "H", np.mean(df[df["id"] == img]["normalise"])))

for img in df[df["type"] == "I"]["id"].unique():
    means.append((img, "I", np.mean(df[df["id"] == img]["normalise"])))
    
df_mean = pd.DataFrame(means)
df_mean.columns = ["id", "type", "mean"]

# Check new dataframe with one image id.
assert np.isclose(np.mean(df[df["id"] == "I01"]["normalise"]), df_mean[df_mean["id"] == "I01"]["mean"])

Boxplot overall modern and historic images.

In [ ]:
g = sns.boxplot(
    x="type",
    y="mean",
    palette=[sns_palette_historic[0], sns_palette_modern[0]],
    data=df_mean
)
set_hline(g)

g.set(title="Niner assessment:\nChoosen mean chroma factor of historic and modern images")
g.figure.set_size_inches(sns_half_plot_size)
g.set(ylim = sns_ylim)
g.set_xlabel("Image type")
g.set_ylabel("Chroma factor mean")
g.set(xticklabels=["Historic", "Modern"])

## t-Test

Select data from all observers and for all images. Devided into the two groups.

In [ ]:
historic = df[df["type"] == "H"]["normalise"]
modern = df[df["type"] == "I"]["normalise"]

assert len(historic) == 30 * num_observers
assert len(modern) == 15 * num_observers

Perform the t-test.
* Null hypothesis: 2 independent samples have identical average (expected) values.
* Our null hypothesis: The samples for historic and modern images have identical average values.

In [ ]:
t_stat, p_value = stats.ttest_ind(historic, modern)

print('P-value:', p_value)
print(f"P-value << 0.05 and reject null hypothesis: {np.less(p_value, 0.05)}.")

Result: We reject the null hypothesis. We have to assume both groups do **not** have identical average expected values.